In [1]:
def project_point_onto_line(x, y, a, b, c):
    # Calculate the line's magnitude
    line_magnitude = (a**2 + b**2)**0.5

    # Normalize the line coefficients
    a_normalized = a / line_magnitude
    b_normalized = b / line_magnitude
    c_normalized = c / line_magnitude

    # Calculate the distance from the point to the line
    distance = a_normalized * x + b_normalized * y + c_normalized

    # Calculate the projected point coordinates
    x_projected = x - a_normalized * distance
    y_projected = y - b_normalized * distance

    return x_projected, y_projected

In [11]:
x1, y1 = project_point_onto_line(1,3,1,0,0)
x2, y2 = project_point_onto_line(2.5,0.5,1,0,0)
print(x1, y1, x2, y2)

0.0 3.0 0.0 0.5


In [12]:
x1, y1 = project_point_onto_line(1,3,0,1,0)
x2, y2 = project_point_onto_line(2.5,0.5,0,1,0)
print(x1, y1, x2, y2)

1.0 0.0 2.5 0.0


In [13]:
X_CONST = 93.1230172983
Y_CONST = 96.3218973211

In [14]:
x1, y1 = project_point_onto_line(1,3,X_CONST,Y_CONST,0)
x2, y2 = project_point_onto_line(2.5,0.5,X_CONST,Y_CONST,0)
print(x1, y1, x2, y2)

-0.982264150517236 0.949642854056187 1.042344341953774 -1.0077277637395903


In [6]:
project_point_onto_line(1,3,0,1,-1)

(1.0, 1.0)

In [3]:
project_point_onto_line(2.5,0.5,0,1,-1)

(2.5, 1.0)